# Elastica paper figures
In this notebook the figures from the paper [Keemink & van Rossum (2015) - etc.) are reproduced, and in the process it is shown how to use the elastica library and some general widgets. 

Necessary libraries:
- matplotlib
- scipy
- numpy
- HoloViews (http://holoviews.org/)

In [ ]:
import holoviews as hv
%load_ext holoviews.ipython
%output widgets='embed' holomap='widgets' size=150 fig='png'
import elastica_neurons as en
import pylab    as pl
from   pylab import exp,cos,sin,pi,tan

First, let's set the basic model parameters

In [ ]:
Ac = 1     # basic tuning magnitude
Kc = 1   # basic tuning width (lower is wider)
a  = 0.1  # modulation strength
E0 = 4     # modulation offset
N  = 32    # number of neurons per location

Define basic plotting functions

In [ ]:
def plotsceneandresponses(scene,iLoc):
    ''' plots the scene and the responses plus underlying drive and modulations for location iLoc
    Inputs
    -----------------
    - scene: the scene to plot
    - iLoc: bar position to plot responses and drives+modulations for
    
    Output
    ------------------
    - Holoviews object of the Figure
    '''
    # get the scene plot
    ScenePlot = scene.plotscene()

    # simulate the scene for location iLoc (i.e. the bar number), and assuming we're not on a torus
    scene.simulate(iLoc,torus='off')

    # get the preferred orientations
    pref        = scene.ang/pi

    # get the drive, modulation, and final response of iLoc
    centerdrive = hv.Curve(zip(pref,scene.FRc[iLoc,:]),label='center drive')
    surroundmod = hv.Curve(zip(pref,scene.FRs[iLoc,:]),label='flanker modulation')
    response    = hv.Curve(zip(pref,scene.FR[iLoc,:]),label='response')

    # make the response plot
    ResponsePlot = surroundmod*centerdrive*response

    # get the modulation contributions
    ModContributions = scene.plotlocalmod(iLoc,torus='off')

    # plot 
    return ScenePlot+ResponsePlot+ModContributions
    

### Example scenes
Setting up a few example scenes. 

3 bars

In [ ]:
%%opts Curve {+axiswise} 
# above makes sure the normalization is done by axis, rather than across all of them

# set bar orientations
a1 = 0*pi/180 # bar 1 orientation
a2 = 45*pi/180 # bar 2 orientation
a3 = 15*pi/180 # bar 3 orientation
A  = pl.array([a1,a2,a3])

# set bar positions (x and y positions)
X = pl.array([[0,0],[-2,-2],[2,2]])

# Setup the scene
scene_3b = en.scene(N,A,X,[15,15],Kc,Ac,a,E0)

# plot
plotsceneandresponses(scene_3b,iLoc=0)

A center bar and n surround bars, with a center and surround orientation. The surround orientation rotates the entire surround.

In [ ]:
%%opts Curve {+axiswise} 
# above makes sure the normalization is done by axis, rather than across all of them
# set some scene parameters
ns = 6 # number of surround bars
R  = 6 # surround distance

# set bar orientations
c = 0*pi/180 # center bar orientation
s = 30*pi/180 # surround bars orientation
A  = pl.zeros(ns+1) # define all bars as having zero degree orientation
A[0] = c # sets first bar as center orientation
A[1:] = pl.ones(ns)*s # sets all other bars as surround orientation

# set bar positions (x and y positions)
X = pl.zeros((ns+1,2)) # set all bars positions to [0,0]
pos = pl.linspace(0,2*pi,ns+1)[:-1]+pi/2 # flanker angular positions (+ pi/2 to get the scene from figure 5F)
X[1:,0]= R*sin(pos) # sets x positions
X[1:,1]= R*cos(pos) # sets y positions
# (position of center is assumed to be [0,0])

# Setup the scene
scene_sur = en.scene(N,A,X,[15,15],Kc,Ac,a,E0,modtype='elastica')

# plot
plotsceneandresponses(scene_sur,iLoc=0)



Calculate the illusion for a given scene, for all flanker orientations

In [ ]:
%%opts Curve {+axiswise} 
# above makes sure the normalization is done by axis, rather than across all of them

# set some scene parameters
ns = 8 # number of surround bars
R  = 6 # surround distance
nO = 32 # number of orientations to test
bias = pl.zeros(nO) # array to store decoding biases in

# set bar orientations
c = 0*pi/180 # center bar orientation
S = pl.linspace(-pi/2,pi/2,nO) # surround bars orientations
A  = pl.zeros(ns+1) # define all bars as having zero degree orientation
A[0] = c # sets first bar as center orientation

# set bar positions (x and y positions)
X = pl.zeros((ns+1,2)) # set all bars positions to [0,0]
pos = pl.linspace(0,2*pi,ns+1)[:-1]+pi/2 # flanker angular positions 
X[1:,0]= R*sin(pos) # sets x positions
X[1:,1]= R*cos(pos) # sets y positions

# set basic scene
scene = en.scene(N,A,X,[15,15],Kc,Ac,a,E0,modtype='elastica')

# loop over surround orientations
for i,s in enumerate(S):
    # reset positions
    pos = pl.linspace(0,2*pi,ns+1)[:-1]+pi/2+s # flanker angular positions , 
    #+s to make the entire surround rotate with surround orientation, remove that to keep bars in place (but still rotate)
    scene.X[1:,0]= R*sin(pos) # resets x positions flankers
    scene.X[1:,1]= R*cos(pos) # resets y positions flankers
    
    # set orientations 
    scene.O[1:] = s # sets all bars except center as surround orientation
    
    # simulate the neurons at the center location
    scene.simulate(0,torus='off')
    
    # get the encoded orientation of the center neuron
    bias[i] = scene.est[0]*180/pi

# plot the scene and biases
scene.plotscene()+hv.Curve(zip(S,bias))


Make a scene with a salient element

In [ ]:
reload(en)

In [ ]:

def fixVScdep_plot(A,X,modtype='distance'):
    # Setup the scene for fixed modulation
    scene_sur = en.scene(N,A,X,[X[:,0].max()+dist,X[:,1].max()+dist],Kc,Ac,a,E0,modtype=modtype)

    # simulate
    scene_sur.simulate_all()

    # calculate saliency
    scene_sur.saliency('mean')

    # get saliencies and orientations
    sal_fix = scene_sur.sal
    or_fix = scene_sur.est

    # do above for cdep
    scene_sur = en.scene(N,A,X,[X[:,0].max()+dist,X[:,1].max()+dist],Kc,Ac,a,E0,modtype=modtype,surdep='cdep')
    scene_sur.simulate_all()
    scene_sur.saliency('mean')
    sal_cdep = scene_sur.sal
    or_cdep = scene_sur.est

    # get maximum saliency
    sal_max = max([max(sal_fix),max(sal_cdep)])

    # plot
    fig = scene_sur.plotscene(length=dist/2) 
    fig+= scene_sur.plotscene(length=dist/2,oriens=or_fix,alphas=sal_fix/sal_max)

    # do above for cdep
    fig+= scene_sur.plotscene(length=dist/2,oriens=or_cdep,alphas=sal_cdep/sal_max)

    # show plot
    return fig

In [ ]:
Ac = 1     # basic tuning magnitude
Kc = 0.6   # basic tuning width (lower is wider)
a  = 3  # modulation strength
E0 = 4     # modulation offset
N  = 32    # number of neurons per location

In [ ]:
%%opts Curve {+axiswise} (linewidth=8)
# above makes sure the normalization is done by axis, rather than across all of them
# set some scene parameters
ns = 9 # width of height of scene in bars
dist = 5 # distance between bars

# set bar orientations
feat = 45*pi/180 # feature bar orientation
back = 0*pi/180 # background bars orientation
A  = pl.ones(ns*ns)*back # define all bars as having backgound degree orientation
feat_ids = [ns*(i)+(i) for i in range(ns)] # choose feature
A[feat_ids] = feat

# set bar positions (x and y positions)
X = pl.array([[x,y] for x in pl.arange(ns)*dist for y in pl.arange(ns)*dist]) # set all bars positions


fig_contour = fixVScdep_plot(A,X)
fig_contour

In [ ]:
%%opts Curve {+axiswise} (linewidth=8)
# above makes sure the normalization is done by axis, rather than across all of them
# set some scene parameters
ns = 9 # width and height of scene in bars
dist = 5 # distance between bars

# set bar orientations
feat = 45*pi/180 # feature bar orientation
back = 0*pi/180 # background bars orientation
A  = pl.ones(ns*ns)*back # define all bars as having backgound degree orientation
feat_ids = ns*ns/2 # choose feature
A[feat_ids] = feat

# set bar positions (x and y positions)
X = pl.array([[x,y] for x in pl.arange(ns)*dist for y in pl.arange(ns)*dist]) # set all bars positions

# plot
fig_single = fixVScdep_plot(A,X)
fig_single

In [ ]:
%%opts Curve {+axiswise} (linewidth=8)
# above makes sure the normalization is done by axis, rather than across all of them
# set some scene parameters
ns = 9 # width and height of scene in bars
dist = 5 # distance between bars

# set bar orientations
feat = 45*pi/180 # feature bar orientation
back = 0*pi/180 # background bars orientation
A  = pl.ones(ns*ns)*back # define all bars as having backgound degree orientation
feat_ids = ns*ns/2 # choose feature
feat_ids = [feat_ids-1-ns,feat_ids-ns,feat_ids+1-ns,feat_ids-1,feat_ids,feat_ids+1,
            feat_ids-1+ns,feat_ids+ns,feat_ids+1+ns]
A[feat_ids] = feat

# set bar positions (x and y positions)
X = pl.array([[x,y] for x in pl.arange(ns)*dist for y in pl.arange(ns)*dist]) # set all bars positions

# plot
fig_block = fixVScdep_plot(A,X)
fig_block

In [ ]:
%%opts Curve {+axiswise} (linewidth=8)
# above makes sure the normalization is done by axis, rather than across all of them
# set some scene parameters
ny = 10 # height in bars
nx = ny*2 # width in bars
dist = 5 # distance between bars
# a = 0.05
# set bar orientations
feat = 0*pi/180 # feature bar orientation
back = 45*pi/180 # background bars orientation
A  = pl.ones(nx*ny)*back # define all bars as having backgound degree orientation
feat_ids = pl.arange(pl.floor(nx*ny/2),nx*ny,dtype=int)
A[feat_ids] = feat

# set bar positions (x and y positions)
X = pl.array([[x,y] for x in pl.arange(nx)*dist for y in pl.arange(ny)*dist]) # set all bars positions

# plot
fig_boundary = fixVScdep_plot(A,X)
fig_boundary

In [ ]:
%%opts Layout [sublabel_format=None tight=True] Overlay [xaxis=None yaxis=None]
fig = fig_single + fig_block + fig_contour + fig_boundary
fig.cols(3)